<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
</center>

<h1> Practico Reglas de Asociacion</h1>
<h2><FONT COLOR="#FF0000"> Autor: Melania Omonte</FONT></h2>
<h3> Importaciones </h3>


In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import scipy as sp
from scipy import stats

from collections import  Counter
from IPython.display import display
from scipy.stats import variation, zscore, kstest, norm

from pylab import *

import warnings
warnings.filterwarnings('ignore')

In [3]:
sns.set_style("whitegrid")
sns.set_context('talk')

In [4]:
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

<h3> Carga de los Datasets </h3>

In [5]:

df_movies = pd.read_csv('movies.csv')
df_ratings = pd.read_csv('ratings.csv')



In [6]:
print('movies -- dimensions: {0};   size: {1}'.format(df_movies.shape, size(df_movies)))
print("Shape = {}".format(df_movies.shape))
df_movies.head()

movies -- dimensions: (27278, 3);   size: 4.50 MB
Shape = (27278, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
print('ratings -- dimensions: {0};   size: {1}'.format(df_ratings.shape, size(df_ratings)))
print("Shape = {}".format(df_ratings.shape))
df_ratings.head()
#df_ratings.sample(10)

ratings -- dimensions: (20000263, 4);   size: 640.01 MB
Shape = (20000263, 4)


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580



<h2>Exploremos un poco los Datasets</h2>

<h3>Movies y Ratings Dataset</h3>


In [8]:
from sklearn.utils import shuffle
movies = shuffle(df_movies).head(int(len(df_movies)/20))
ratings = shuffle(df_ratings).head(int(len(df_ratings)/20))
print('movies -- dimensions: {0};   size: {1}'.format(df_movies.shape, size(df_movies)))
print('ratings -- dimensions: {0};   size: {1}'.format(df_ratings.shape, size(df_ratings)))

movies -- dimensions: (27278, 3);   size: 4.50 MB
ratings -- dimensions: (20000263, 4);   size: 640.01 MB


<h3> Unimos los dataset con el Id en comun "moviedID"</h3>

In [9]:
merge = df_ratings.merge(df_movies, on = 'movieId', how = 'left')

<h3>Mostramos la union </h3>

In [10]:
##print("Shape = {}".format(merge.shape))

In [11]:
display(merge.head())

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,1112484676,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,1112484819,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,1112484727,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,1112484580,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [12]:
merge=merge.sort_values( by='movieId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
merged=merge.values[:,[0,2]]
print(merged)

[[97809 3.0]
 [106140 5.0]
 [106138 3.0]
 ...
 [28906 2.5]
 [65409 3.0]
 [133047 4.0]]


<h3> Teams Dataset</h3>

In [13]:
transactions = merge.groupby('userId')['movieId'].apply(list).values

In [39]:
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.25,  min_confidence=0.6)

In [40]:
#Rule.lift
rules=sorted(rules, key=lambda rule: rule.lift, reverse=True)
for rule in rules:
  print(rule)


{1210} -> {1196} (conf: 0.765, supp: 0.259, lift: 2.338, conv: 2.864)
{1196} -> {1210} (conf: 0.791, supp: 0.259, lift: 2.338, conv: 3.164)
{1196} -> {260} (conf: 0.859, supp: 0.281, lift: 2.182, conv: 4.291)
{260} -> {1196} (conf: 0.714, supp: 0.281, lift: 2.182, conv: 2.352)
{1210} -> {260} (conf: 0.837, supp: 0.283, lift: 2.128, conv: 3.730)
{260} -> {1210} (conf: 0.720, supp: 0.283, lift: 2.128, conv: 2.361)
{377} -> {480} (conf: 0.848, supp: 0.254, lift: 1.966, conv: 3.740)
{589} -> {480} (conf: 0.791, supp: 0.298, lift: 1.834, conv: 2.717)
{480} -> {589} (conf: 0.692, supp: 0.298, lift: 1.834, conv: 2.020)
{356, 593} -> {480} (conf: 0.786, supp: 0.252, lift: 1.823, conv: 2.658)
{296, 480} -> {356} (conf: 0.867, supp: 0.261, lift: 1.815, conv: 3.935)
{480, 593} -> {356} (conf: 0.859, supp: 0.252, lift: 1.799, conv: 3.717)
{296, 356} -> {480} (conf: 0.775, supp: 0.261, lift: 1.798, conv: 2.529)
{480} -> {296, 356} (conf: 0.604, supp: 0.261, lift: 1.798, conv: 1.677)
{592} -> {480} 

In [66]:
rules=sorted(rules, key=lambda rule: rule.confidence)
for rule in rules:
  print(rule) # Prints the rule and its confidence, support, lift, ...

{356} -> {110} (conf: 0.602, supp: 0.288, lift: 1.550, conv: 1.536)
{480} -> {296, 356} (conf: 0.604, supp: 0.261, lift: 1.798, conv: 1.677)
{480} -> {318} (conf: 0.614, supp: 0.265, lift: 1.341, conv: 1.404)
{296} -> {480} (conf: 0.618, supp: 0.300, lift: 1.433, conv: 1.489)
{480} -> {110} (conf: 0.624, supp: 0.269, lift: 1.607, conv: 1.626)
{480} -> {457} (conf: 0.633, supp: 0.273, lift: 1.769, conv: 1.752)
{356} -> {318} (conf: 0.640, supp: 0.306, lift: 1.398, conv: 1.505)
{593} -> {480} (conf: 0.641, supp: 0.293, lift: 1.486, conv: 1.584)
{318} -> {356} (conf: 0.668, supp: 0.306, lift: 1.398, conv: 1.573)
{356} -> {593} (conf: 0.670, supp: 0.320, lift: 1.467, conv: 1.647)
{110} -> {318} (conf: 0.671, supp: 0.260, lift: 1.466, conv: 1.648)
{318} -> {593} (conf: 0.679, supp: 0.311, lift: 1.485, conv: 1.691)
{480} -> {593} (conf: 0.679, supp: 0.293, lift: 1.486, conv: 1.693)
{593} -> {318} (conf: 0.680, supp: 0.311, lift: 1.485, conv: 1.693)
{296} -> {318} (conf: 0.680, supp: 0.330, l

In [67]:

filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
  print(rule)

{1210} -> {1196} (conf: 0.765, supp: 0.259, lift: 2.338, conv: 2.864)
{1196} -> {1210} (conf: 0.791, supp: 0.259, lift: 2.338, conv: 3.164)
{260} -> {1196} (conf: 0.714, supp: 0.281, lift: 2.182, conv: 2.352)
{1196} -> {260} (conf: 0.859, supp: 0.281, lift: 2.182, conv: 4.291)
{260} -> {1210} (conf: 0.720, supp: 0.283, lift: 2.128, conv: 2.361)
{1210} -> {260} (conf: 0.837, supp: 0.283, lift: 2.128, conv: 3.730)
{377} -> {480} (conf: 0.848, supp: 0.254, lift: 1.966, conv: 3.740)
{480} -> {589} (conf: 0.692, supp: 0.298, lift: 1.834, conv: 2.020)
{589} -> {480} (conf: 0.791, supp: 0.298, lift: 1.834, conv: 2.717)
{592} -> {480} (conf: 0.770, supp: 0.256, lift: 1.785, conv: 2.471)
{480} -> {457} (conf: 0.633, supp: 0.273, lift: 1.769, conv: 1.752)
{457} -> {480} (conf: 0.763, supp: 0.273, lift: 1.769, conv: 2.400)
{377} -> {356} (conf: 0.834, supp: 0.250, lift: 1.746, conv: 3.147)
{47} -> {296} (conf: 0.836, supp: 0.261, lift: 1.719, conv: 3.128)
{150} -> {480} (conf: 0.726, supp: 0.250,

<h3>Matchs Dataset</h3>

In [68]:

filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
    movie1 = df_movies[df_movies.movieId == rule.lhs].title.item()
    movie2 = df_movies[df_movies.movieId == rule.rhs].title.item()
    print(movie1, " -> ", movie2)

Star Wars: Episode VI - Return of the Jedi (1983)  ->  Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode V - The Empire Strikes Back (1980)  ->  Star Wars: Episode VI - Return of the Jedi (1983)
Star Wars: Episode IV - A New Hope (1977)  ->  Star Wars: Episode V - The Empire Strikes Back (1980)
Star Wars: Episode V - The Empire Strikes Back (1980)  ->  Star Wars: Episode IV - A New Hope (1977)
Star Wars: Episode IV - A New Hope (1977)  ->  Star Wars: Episode VI - Return of the Jedi (1983)
Star Wars: Episode VI - Return of the Jedi (1983)  ->  Star Wars: Episode IV - A New Hope (1977)
Speed (1994)  ->  Jurassic Park (1993)
Jurassic Park (1993)  ->  Terminator 2: Judgment Day (1991)
Terminator 2: Judgment Day (1991)  ->  Jurassic Park (1993)
Batman (1989)  ->  Jurassic Park (1993)
Jurassic Park (1993)  ->  Fugitive, The (1993)
Fugitive, The (1993)  ->  Jurassic Park (1993)
Speed (1994)  ->  Forrest Gump (1994)
Seven (a.k.a. Se7en) (1995)  ->  Pulp Fiction (1994)
Ap

In [59]:
filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
    movie1 = df_movies[df_movies.movieId == rule.lhs].genres.item()
    movie2 = df_movies[df_movies.movieId == rule.rhs].genres.item()
    print(movie1, " -> ", movie2)

Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Romance|Thriller  ->  Action|Adventure|Sci-Fi|Thriller
Action|Sci-Fi  ->  Action|Adventure|Sci-Fi|Thriller
Action|Adventure|Sci-Fi|Thriller  ->  Action|Sci-Fi
Action|Crime|Thriller  ->  Action|Adventure|Sci-Fi|Thriller
Action|Adventure|Sci-Fi|Thriller  ->  Thriller
Thriller  ->  Action|Adventure|Sci-Fi|Thriller
Action|Romance|Thriller  ->  Comedy|Drama|Romance|War
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Adventure|Drama|IMAX  ->  Action|Adventure|Sci-Fi|Thriller
Action|Adventure|Sci-Fi|Thriller  ->  Comedy|Drama|Romance|War
Comedy|Drama|Romance|War  ->  Action|Adventure|Sci-Fi|Thriller
Thriller  ->  Crime|Horror|Thriller
Crime|Mystery|Thriller  ->  Crime|Dra

In [64]:

filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1 and (rule.lift) >= 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
    movie1 = df_movies[df_movies.movieId == rule.lhs].title.item()
    movie2 = df_movies[df_movies.movieId == rule.rhs].title.item()
    saga1 = movie1.split()
    saga2 = movie2.split()
    
    counts = 0
    for word in s1:
        if word in s2:
            counts = counts + 1
    
    if counts < 2:
        print(m1, " -> ", m2)

Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller


In [65]:
# filtramos las reglas 1-1, con lift >= 1.7, y que tienen menos de dos palabras en común. 
# Las imprimimos ordenadas por lift y decodificadas.
filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1 and (rule.lift) >= 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
    movie1 = df_movies[df_movies.movieId == rule.lhs].genres.item()
    movie2 = df_movies[df_movies.movieId == rule.rhs].genres.item()
    saga1 = movie1.split()
    saga2 = movie2.split()
    
    counts = 0
    for word in saga1:
        if word in saga2:
            counts = counts + 1
    
    if counts < 2:
        print(movie1, " -> ", movie2)

Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Adventure|Sci-Fi  ->  Action|Adventure|Sci-Fi
Action|Romance|Thriller  ->  Action|Adventure|Sci-Fi|Thriller
Action|Sci-Fi  ->  Action|Adventure|Sci-Fi|Thriller
Action|Adventure|Sci-Fi|Thriller  ->  Action|Sci-Fi
Action|Crime|Thriller  ->  Action|Adventure|Sci-Fi|Thriller
Action|Adventure|Sci-Fi|Thriller  ->  Thriller
Thriller  ->  Action|Adventure|Sci-Fi|Thriller
Action|Romance|Thriller  ->  Comedy|Drama|Romance|War
Mystery|Thriller  ->  Comedy|Crime|Drama|Thriller
Adventure|Drama|IMAX  ->  Action|Adventure|Sci-Fi|Thriller
Action|Adventure|Sci-Fi|Thriller  ->  Comedy|Drama|Romance|War
Comedy|Drama|Romance|War  ->  Action|Adventure|Sci-Fi|Thriller
Thriller  ->  Crime|Horror|Thriller
Crime|Mystery|Thriller  ->  Crime|Dra

<h3>Conclusiones</h3>


Como soporte mínimo un valor de 25% y una confianza minima un valor de 60%, podemos observar:

Algunas recomendaciones contienen películas que forman parte de una misma saga.

Si necesitariamos un mayor número de recomendaciones,deberiamos disminuir el valor del soporte minimo o la confianza mínima requerida.